#### Below is the retraining of the SINCCONV with only 34 keywords. The ODL implementation can be found under the 'On-Line Learning' section below

In [ ]:
# !pip install torchinfo
# !pip install torchaudio

In [ ]:
import torchaudio
import torch
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import sys
from torch.autograd import Variable
import math

from torchinfo import summary
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import IPython.display as ipd
import os
import soundfile as sf
from os import listdir
from os.path import isdir, join
import pandas as pd
import pathlib
import torch.optim as optim
import tensorflow as tf

from tqdm.notebook import tqdm

In [ ]:
#should be  >1.2
#pip install --upgrade pandas
# pd.__version__

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# device ='cpu'
print(device)

In [ ]:
# data_dir = pathlib.Path('/content/data')
# if not data_dir.exists():
#   tf.keras.utils.get_file(
#       'speech_commands_v0.02.tar.gz',
#       origin="http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz",
#       extract=True,
#       cache_dir='.', cache_subdir='data')

In [ ]:
# output_folder =str(output_folder)

In [ ]:
data_dir = 'C:/Users/cferr/Documents/4th Year/FYP Data/speech_commands_v0.02/'

In [ ]:
keywords = [name for name in listdir(data_dir) if isdir(join(data_dir, name))]
#remove bg noise as it not needed and requires extra pre-processing
keywords.remove('_background_noise_')
keywords.remove('zero')
print(keywords)

In [ ]:
word2index = {
    # core words
    "backward": 0,
    "bed": 1,
    "bird": 2,
    "cat": 3,
    "dog": 4,
    "down": 5,
    "eight": 6,
    "five": 7,
    "follow": 8,
    "forward": 9,
    "four": 10,
    "go": 11,
    "happy": 12,
    "house": 13,
    "learn": 14,
    "left": 15,
    "marvin": 16,
    "nine": 17,
    "no": 18,
    "off": 19,
    "on":20,
    "one":21,
    "right":22,
    "seven":23,
    "sheila":24,
    "six":25,
    "stop":26,
    "three":27,
    "tree":28,
    "two":29,
    "up":30,
    "visual":31,
    "wow":32,
    "yes":33,
}

index2word = [word for word in word2index]

In [ ]:
filenames = []
y = []
for word_class in word2index:
    for files in listdir(join(data_dir, word_class)):
        filenames.append(join(word_class, files))
        y.append(word2index[word_class]) 

In [ ]:
#create a dictionary of the filenames and labels
combined_dict = dict(zip(filenames,y))
#save that dictionary
np.save('files_dict', combined_dict)

In [ ]:
train_data, validation_data, train_classes, validation_classes = train_test_split(filenames, y,
                                                                      test_size=0.2, random_state=42, shuffle=True)

In [ ]:
def getLists(files_in, output_name):
  MyFile=open(output_name,'w')

  for element in files_in:
      MyFile.write(element)
      MyFile.write('\n')
  MyFile.close()

In [ ]:
getLists(filenames, 'fileslist.scp')
getLists(validation_data, 'test.scp')
getLists(train_data, 'train.scp')

In [ ]:
data_folder = data_dir
outdim = len(keywords)
lab_dict = np.load('files_dict.npy', allow_pickle=True).item()

lr=0.001
batch_size=128
Batch_dev=128
N_epochs=100
N_batches=665
N_eval_epoch=8
seed=42

torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
def ReadList(list_file):
 f=open(list_file,"r")
 lines=f.readlines()
 list_sig=[]
 for x in lines:
    list_sig.append(x.rstrip())
 f.close()
 return list_sig

In [ ]:
wav_lst_tr=ReadList('train.scp')
snt_tr=len(wav_lst_tr)
# test list
wav_lst_te=ReadList('test.scp')
snt_te=len(wav_lst_te)

In [ ]:
wlen=16000

### SincConv module

In [ ]:
import numpy as np
import math

class SincConv(torch.nn.Module):
    @staticmethod
    def to_mel(hz):
        return 2595 * np.log10(1 + hz / 700)

    @staticmethod
    def to_hz(mel):
        return 700 * (10 ** (mel / 2595) - 1)

    def __init__(self, out_channels, kernel_size, sample_rate=16000, in_channels=1, stride=1, padding=0, min_low_hz=50, min_band_hz=50):
        super(SincConv,self).__init__()

        if in_channels != 1:
            msg = "SincConv only support one input channel (here, in_channels = {%i})" % (in_channels)
            raise ValueError(msg)

        self.out_channels = out_channels
        self.kernel_size = kernel_size
        # Forcing the filters to be odd (i.e, perfectly symmetrics)
        if kernel_size%2==0:
            self.kernel_size=self.kernel_size+1
        # parameters    
        self.stride = stride
        self.padding = padding
        self.sample_rate = sample_rate
        self.min_low_hz = min_low_hz
        self.min_band_hz = min_band_hz
        # initialize filterbanks such that they are equally spaced in Mel scale
        low_hz = 30
        high_hz = self.sample_rate / 2 - (self.min_low_hz + self.min_band_hz)
        mel = np.linspace(self.to_mel(low_hz), self.to_mel(high_hz), self.out_channels + 1)
        hz = self.to_hz(mel)
        # filter lower frequency (out_channels, 1)
        self.low_hz_ = torch.nn.Parameter(torch.Tensor(hz[:-1]).view(-1, 1))
    
        #
        # self.low_hz_ =torch.Tensor(hz[:-1]).view(-1, 1).to(device)
        # filter frequency band (out_channels, 1)
        self.band_hz_ = torch.nn.Parameter(torch.Tensor(np.diff(hz)).view(-1, 1))
        # self.band_hz_ =torch.Tensor(np.diff(hz)).view(-1, 1).to(device)

        # Hamming window
        n_lin=torch.linspace(0, (self.kernel_size/2)-1, steps=int((self.kernel_size/2))) # computing only half of the window
        self.window_=0.54-0.46*torch.cos(2*math.pi*n_lin/self.kernel_size);
        # (1, kernel_size/2)
        n = (self.kernel_size - 1) / 2.0
        self.n_ = 2*math.pi*torch.arange(-n, 0).view(1, -1) / self.sample_rate # Due to symmetry, I only need half of the time axes

    def forward(self, waveforms):
        self.n_ = self.n_.to(waveforms.device)
        self.window_ = self.window_.to(waveforms.device)
        low = self.min_low_hz  + torch.abs(self.low_hz_)
        high = torch.clamp(low + self.min_band_hz + torch.abs(self.band_hz_),self.min_low_hz,self.sample_rate/2)
        band=(high-low)[:,0]
        
        f_times_t_low = torch.matmul(low, self.n_)
        f_times_t_high = torch.matmul(high, self.n_)

        band_pass_left=((torch.sin(f_times_t_high)-torch.sin(f_times_t_low))/(self.n_/2))*self.window_ # Equivalent of Eq.4 of the reference paper (SPEAKER RECOGNITION FROM RAW WAVEFORM WITH SINCNET). I just have expanded the sinc and simplified the terms. This way I avoid several useless computations. 
        band_pass_center = 2*band.view(-1,1)
        band_pass_right= torch.flip(band_pass_left,dims=[1])
        
        band_pass=torch.cat([band_pass_left,band_pass_center,band_pass_right],dim=1)
        band_pass = band_pass / (2*band[:,None])

        self.filters = (band_pass).view(self.out_channels, 1, self.kernel_size)
        return torch.nn.functional.conv1d(waveforms, self.filters, stride=self.stride, padding=self.padding, dilation=1, bias=None, groups=1) 

### LogAbs Acitvation Module

In [ ]:
class LogAbs(torch.nn.Module):
    def __init__(self):
        super(LogAbs,self).__init__()

    def forward(self, x):
        return torch.log(torch.abs(x) + 1)

### Sinc and Separable Conv Blocks

In [ ]:
def SincConvBlock(c, k, s, sample_rate=16000):
    sinc_conv = SincConv(out_channels=c, kernel_size=k, sample_rate=sample_rate, in_channels=1, stride=s, padding=k//2)
    avg_pool = torch.nn.AvgPool1d(kernel_size=2, stride=2)
    return torch.nn.Sequential(sinc_conv, LogAbs(), torch.nn.BatchNorm1d(c), avg_pool)

def DSConvBlock(c_in, c, k, s):
    depth_conv = torch.nn.Conv1d(in_channels=c_in, out_channels=c_in, kernel_size=k, stride=s, padding=k//2, groups=c_in)
    point_conv = torch.nn.Conv1d(in_channels=c_in, out_channels=c, kernel_size=1, stride=1)
    return torch.nn.Sequential(depth_conv, point_conv, torch.nn.ReLU(), torch.nn.BatchNorm1d(c), torch.nn.AvgPool1d(2), torch.nn.Dropout(0.1))

In [ ]:
def batch_gen(batch_size,data_folder,wav_lst,N_snt,wlen,lab_dict):
    sig_batch=torch.zeros([batch_size,1, wlen])
    lab_batch=torch.zeros(batch_size)
    snt_id_arr=torch.randint(0, N_snt, (batch_size,))
    
    waveforms=[]
    for i in range(batch_size):
        waveform, sample_rate = torchaudio.load(str(data_folder)+'/'+wav_lst[snt_id_arr[i]])

        #ensure all waveforms are padded to a length of 16000
        snt_len=waveform.shape[1]
        if(snt_len < 16000):
            target = torch.zeros((1,16000))
            target[:, :snt_len] = waveform
            waveform=target 
        waveforms.append(waveform)
        lab_batch[i]=lab_dict[wav_lst[snt_id_arr[i]]]
    s = torch.cat(waveforms).unsqueeze(1) # Audio samples, Feature first (1 since mono)
    return s, lab_batch

Test the batch_gen function to ensure its working as intended

In [ ]:
[inp,lab]=batch_gen(3,data_folder,wav_lst_tr,snt_tr,wlen,lab_dict)

### Combine all elements of the model as seen in Mittermaier et. al

In [ ]:
sinc_conv_model = torch.nn.Sequential(
    SincConvBlock(c=40, k=101, s=8),
    DSConvBlock(40,160,25,2), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    torch.nn.AvgPool1d(15),
    torch.nn.Flatten(),
    torch.nn.Linear(160,35),
    torch.nn.Softmax(dim=1)
)

In [ ]:
print(sinc_conv_model)

In [ ]:
[inp,lab]=batch_gen(1,data_folder,wav_lst_tr,snt_tr,wlen,lab_dict)
sinc_conv_model=sinc_conv_model.to(device)
inp = inp.to(device)
summary(sinc_conv_model, input_data=[inp])

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
sinc_model=sinc_conv_model.to(device)

In [ ]:
n = count_parameters(sinc_model)
print("Number of parameters: %s" % n)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
%load_ext tensorboard

In [ ]:
cost = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(sinc_model.parameters(), lr=lr, eps=1e-8)
log_interval = 800

In [ ]:
def train(model, epoch, log_interval):
    # model=model.to(device)
    model.train()
      
    for i in range(N_batches):
        [inp,target]=batch_gen(batch_size,data_folder,wav_lst_tr,snt_tr,wlen,lab_dict)

        inp = inp.to(device)
        

        target = target.type(torch.LongTensor)
        target = target.to(device)
        output = model(inp)
          # print(type(output))
          # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = cost(output, target)
        writer.add_scalar("Loss/train", loss, epoch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

          # print training stats
        if i % log_interval == 0:
              print(f"Train Epoch: {epoch} [{i * len(inp)}/{snt_tr} ({100. * i / N_batches:.0f}%)]\tLoss: {loss.item():.6f}")

          # update progress bar
        pbar.update(pbar_update)
          # record loss
        losses.append(loss.item())

In [ ]:
N_batches_te=170

In [ ]:
def test(model, epoch):
    model.eval()
    correct = 0
    for i in range(N_batches_te):
        [data,target]=batch_gen(batch_size,data_folder,wav_lst_te,snt_te,wlen,lab_dict)
        data = data.to(device)
        target = target.to(device)

        output = model(data)

        pred = pred=torch.max(output,dim=1)[1]
        
        correct+= (pred == target).float().sum() 

        # update progress bar
        pbar.update(pbar_update)
    accuracy.append(correct/(batch_size*N_batches_te))
    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{(batch_size*N_batches_te)} ({100. * correct /snt_te:.2f}%)\n")

In [ ]:
#controls how often training updates are printed on the screen
log_interval = 300
n_epoch = 22

pbar_update = 1 / snt_te+snt_tr
losses = []
accuracy=[]

with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(sinc_model, epoch, log_interval)
        test(sinc_model, epoch)

writer.close()

In [ ]:
%tensorboard --logdir runs

In [ ]:
torch.save(sinc_model.state_dict(), 'Trained_Models/model_34.pth')

## On-line Learning / One-shot Learning

In [ ]:
feature_mapping_model = torch.nn.Sequential(
    SincConvBlock(c=40, k=101, s=8),
    DSConvBlock(40,160,25,2), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    DSConvBlock(160,160,9,1), 
    torch.nn.AvgPool1d(15),
    torch.nn.Flatten())

In [ ]:
model_tmp = sinc_conv_model
model_tmp.load_state_dict(torch.load('Trained_Models/model_34.pth', map_location=torch.device('cpu')))

In [ ]:
feature_mapping_model = torch.nn.Sequential(*[model_tmp[i] for i in range(8)])

In [ ]:
feature_mapping_model = feature_mapping_model.to('cpu')

In [ ]:
feature_mapping_model(inp).cpu().detach().numpy()

Next step is to create vectors for all the existing keywords but also for the unseen keyword of 'zero' <br> Then need to create a Y vector where every keyword but 'zero' has a value of 0

In [ ]:
def batch_gen_zero(batch_size,data_folder,wav_lst,N_snt,wlen,lab_dict):
    sig_batch=torch.zeros([batch_size,1, wlen])
    lab_batch=torch.zeros(batch_size)
    snt_id_arr=torch.arange(0, N_snt)
    
    waveforms=[]
    for i in range(batch_size):
        waveform, sample_rate = torchaudio.load(str(data_folder)+'/'+wav_lst[snt_id_arr[i]])

        #ensure all waveforms are padded to a length of 16000
        snt_len=waveform.shape[1]
        if(snt_len < 16000):
            target = torch.zeros((1,16000))
            target[:, :snt_len] = waveform
            waveform=target 
        waveforms.append(waveform)
    s = torch.cat(waveforms).unsqueeze(1) # Audio samples, Feature first (1 since mono)
    return s, lab_batch

In [ ]:
zero_dict = {'zero':1}

In [ ]:
filename_zero = []
y_zero = []
for word_class in zero_dict:
    for files in listdir(join(data_dir, word_class)):
        filename_zero.append(join(word_class, files))
        y_zero.append(zero_dict[word_class]) 

In [ ]:
zero_train = filename_zero[:-800]
zero_test = filename_zero[-800:]
zero_train_y = y_zero[:-800]
zero_test_y = y_zero[-800:]

In [ ]:
def make_matrix(filelist):
  temp =[]
  for i in range(len(filelist)):
    inp, _ = batch_gen_zero(1,data_folder,filelist, len(filelist),wlen, lab_dict)
    inp = inp.to('cpu')

    temp.append(feature_mapping_model(inp).cpu().detach().numpy())

  temp = np.array(temp)
  temp = temp.squeeze(1)

  return temp

In [ ]:
wav_list_reg = wav_lst_tr[:4000]

In [ ]:
full_matrix = make_matrix(wav_list_reg)
zero_matrix = make_matrix(filename_zero)
full_matrix = np.append(full_matrix, zero_matrix,axis=0)

In [ ]:
# scaled_matrix = scaler.fit_transform(full_matrix)
scaled_matrix= full_matrix

In [ ]:
accuracies =[]
sample_size =[]

#we incrementally add 100 additional samples of 'zero' at each iteration
# at each interation a new regression model is created and its score evaluated and stored
for i in range(4100, len(full_matrix), 100):
    resampled_matrix =[]
    resampled_matrix = full_matrix[-i:]
    resampled_y = y[-i:]
    
    train_matrix, test_matrix, train_classes, test_classes = train_test_split(resampled_matrix, resampled_y, test_size=0.2,
                                                                              random_state=42, shuffle=True)
    reg= LinearRegression(normalize=True)
    reg.fit(train_matrix, train_classes)
    accuracy = (reg.score(test_matrix, test_classes))
    sample_size.append(i)
    accuracies.append(accuracy)

In [ ]:
df = pd.DataFrame(columns=['Accuracy', 'No_Samples'])

In [ ]:
df['Accuracy'] = accuracies
df['No_Samples'] = sample_size

In [ ]:
df.plot(x='No_Samples',y='Accuracy',xlabel='Total Number Samples', ylabel='Accuracy', grid=True, figsize=(12,8), 
        title='Accuracy vs Samples - Linear Regression', legend=False)

In [ ]:
resampled_matrix =[]
resampled_matrix = full_matrix[-6100:]
resampled_y = y[-6100:]
    
train_matrix, test_matrix, train_classes, test_classes = train_test_split(resampled_matrix, resampled_y, test_size=0.2,
                                                                              random_state=42, shuffle=True)

In [ ]:
reg= LinearRegression(normalize=True)
reg.fit(train_matrix, train_classes)
accuracy = (reg.score(test_matrix, test_classes))

In [ ]:
accuracy